In [ ]:
import os
import json
import requests
import time 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime

from functions import load_api_key, plotting

pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

In [ ]:
household_supplies = pd.read_excel('inputs/outputFile.xlsx', sheet_name = 'T1')
household_supplies.columns = household_supplies.iloc[9]
household_supplies = household_supplies.iloc[10:19]
household_supplies = household_supplies.reset_index(drop = True)

In [ ]:
HDB_supply = household_supplies.iloc[:5]
private_supply = household_supplies.iloc[6:8]

In [ ]:
total_HDB_supply = HDB_supply.sum(axis = 0).to_frame().reset_index().iloc[1:]
total_HDB_supply.columns = ['year', 'total_HDB_supply']
total_HDB_supply.head()

In [ ]:
total_private_supply = private_supply.sum(axis = 0).to_frame().reset_index().iloc[1:]
total_private_supply.columns = ['year', 'total_private_supply']
total_private_supply.head()

In [ ]:
total_HDB_supply['year'] = total_HDB_supply['year'].astype(int)
total_HDB_supply.loc[0, 'year'] = 2024
total_HDB_supply = total_HDB_supply.sort_values('year', ascending = False)
total_HDB_supply['total_HDB_supply_prev_year'] = total_HDB_supply['total_HDB_supply'].shift(-1)

total_private_supply['year'] = total_private_supply['year'].astype(int)
total_private_supply.loc[0, 'year'] = 2024
total_private_supply = total_private_supply.sort_values('year', ascending = False)
total_private_supply['total_private_supply_prev_year'] = total_private_supply['total_private_supply'].shift(-1)

In [ ]:
total_HDB_supply

In [ ]:
total_HDB_supply = total_HDB_supply.sort_values('year')
total_HDB_supply['percent_change'] = total_HDB_supply['total_HDB_supply'].pct_change() * 100
total_HDB_supply['category'] = 'HDB'

total_private_supply = total_private_supply.sort_values('year')
total_private_supply['percent_change'] = total_private_supply['total_private_supply'].pct_change() * 100
total_private_supply['category'] = 'private'


In [ ]:
total_supply = pd.concat([total_HDB_supply[['year', 'category', 'percent_change']], total_private_supply[['year', 'category', 'percent_change']]])

In [ ]:
df = total_supply.loc[total_supply['year'].between(2022,2024)]
plotting('Unit supply growth in each sector', 'barplot', df, 'category', 'percent_change', 'year')

In [ ]:
rental_data_df = pd.read_csv('inputs/rental_with_coordinates.csv')
rental_data_df = rental_data_df.merge(total_HDB_supply, on = 'year', how = 'left')
rental_data_df = rental_data_df.merge(total_private_supply, on = 'year', how = 'left')
rental_data_df.head()

In [ ]:
total_rents_df = rental_data_df.groupby('year').size().reset_index()
total_rents_df = total_rents_df.rename(columns = {0 : 'count_of_rents'})
total_rents_df = total_rents_df.merge(total_HDB_supply, on = 'year', how = 'left')
total_rents_df = total_rents_df.merge(total_private_supply, on = 'year', how = 'left')
total_rents_df['HDB_rent_supply_ratio'] = total_rents_df['count_of_rents'] / total_rents_df['total_HDB_supply']
total_rents_df['private_rent_supply_ratio'] = total_rents_df['count_of_rents'] / total_rents_df['total_private_supply']
total_rents_df['HDB_rent_prev_supply_ratio'] = total_rents_df['count_of_rents'] / total_rents_df['total_HDB_supply_prev_year']
total_rents_df['private_rent_prev_supply_ratio'] = total_rents_df['count_of_rents'] / total_rents_df['total_private_supply_prev_year']
total_rents_df